In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings

from futsim_funcs import TPR,TFR,TPSR,TGKR,TDR

In [2]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

<br>

In [3]:
players_df = pd.read_csv("../fifa24_db/pdb_23.csv")
players_df = players_df

# futsim_funcs.py

In [4]:
def TPR(df, top_N_players=18):
    '''
    Team Power Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        top_players = club_data.sort_values(by='overall', ascending=False).head(top_N_players)
        mean_top_players = top_players.mean(numeric_only=True)
        
        goalkeepers = top_players[top_players['player_positions'].str.contains('GK')].mean(numeric_only=True).fillna(70)
        defenders = top_players[top_players['player_positions'].str.contains('CB|LB|RB|LWB|RWB')].mean(numeric_only=True).fillna(70)
        midfielders = top_players[top_players['player_positions'].str.contains('CM|CDM|CAM|LM|RM')].mean(numeric_only=True).fillna(70)
        attackers = top_players[top_players['player_positions'].str.contains('ST|CF|LW|RW|LF|RF')].mean(numeric_only=True).fillna(70)

        club_df = pd.DataFrame({
            "league_id": top_players.iloc[0]['league_id'],
            "club_team_id": top_players.iloc[0]['club_team_id'],
            "league_name": top_players.iloc[0]['league_name'],
            "club_name": club_name,
            "def_power": int(goalkeepers['overall']*0.2 + defenders['overall']*0.8),
            "mid_power": int(midfielders['overall']),
            "att_power": int(attackers['overall']),
            "power": int(mean_top_players['overall']),
        }, index=[0])
        
        attributes = [
            'attacking_crossing','attacking_finishing', 'attacking_heading_accuracy',
           'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
           'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
           'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
           'movement_agility', 'movement_reactions', 'movement_balance',
           'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
           'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
           'mentality_positioning', 'mentality_vision', 'mentality_penalties',
           'mentality_composure', 'defending_marking_awareness',
           'defending_standing_tackle', 'defending_sliding_tackle',
           'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
           'goalkeeping_positioning', 'goalkeeping_reflexes'
        ]

        mean_data = {}
        for position, players in zip(['goalkeepers', 'defenders', 'midfielders', 'attackers'], [goalkeepers, defenders, midfielders, attackers]):
            for attribute in attributes:
                mean_data[f"{position}_{attribute}"] = int(players[attribute]) if not pd.isnull(players[attribute]) else 70

        mean_df = pd.DataFrame(mean_data, index=[0]) 
        club_df = pd.concat([club_df, mean_df], axis=1)
        
        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "power",
        "data": merged.sort_values("power", ascending=False)
    }

In [5]:
def TFR(tpr_data, df, top_N_players=18):
    '''
    Team Finishing Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        top_players = club_data.sort_values(by='overall', ascending=False).head(top_N_players)
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")

        power = tpr_data_club["power"].iloc[0]
        att = tpr_data_club["att_power"].iloc[0]
        aaf = tpr_data_club["attackers_attacking_finishing"].iloc[0]
        mpsp = tpr_data_club["midfielders_power_shot_power"].iloc[0]
        mls = tpr_data_club["midfielders_power_long_shots"].iloc[0]
        mha = tpr_data_club["defenders_attacking_heading_accuracy"].iloc[0]
            
        skill_rating = int(power * 0.35 +
                               att * 0.15 +
                               aaf * 0.15 +
                               mpsp * 0.15 +
                               mls * 0.15 +
                               mha * 0.05)

        club_df = pd.DataFrame({
            "league_id": top_players.iloc[0]['league_id'],
            "club_team_id": top_players.iloc[0]['club_team_id'],  
            "league_name": top_players.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "finishing": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "finishing",
        "data": merged.sort_values("finishing", ascending=False)
    }

In [6]:
def TPSR(tpr_data, df, top_N_players=18):
    '''
    Team Passing Rating
    '''
    club_groups = df.groupby('club_name')
    dfs = []
    for club_name, club_data in club_groups:
        top_players = club_data.sort_values(by='overall', ascending=False).head(top_N_players)
        tpr_data_club  = tpr_data.query(f"club_name == '{club_name}'")

        power = tpr_data_club["power"].iloc[0]
        att = tpr_data_club["att_power"].iloc[0]
        aaf = tpr_data_club["attackers_attacking_finishing"].iloc[0]
        mpsp = tpr_data_club["midfielders_power_shot_power"].iloc[0]
        mls = tpr_data_club["midfielders_power_long_shots"].iloc[0]
        mha = tpr_data_club["defenders_attacking_heading_accuracy"].iloc[0]
            
        skill_rating = int(power * 0.35 +
                               att * 0.15 +
                               aaf * 0.15 +
                               mpsp * 0.15 +
                               mls * 0.15 +
                               mha * 0.05)

        club_df = pd.DataFrame({
            "league_id": top_players.iloc[0]['league_id'],
            "club_team_id": top_players.iloc[0]['club_team_id'],  
            "league_name": top_players.iloc[0]['league_name'],
            "club_name": club_name,
            "power": power,
            "passing": skill_rating
        }, index=[0])

        dfs.append(club_df)
    merged = pd.concat(dfs, ignore_index=True)

    return {
        "name": "passing",
        "data": merged.sort_values("passing", ascending=False)
    }    

# New Rating Functions

In [10]:
tpr_data = TPR(players_df, top_N_players=18)["data"]
tpr_data.sort_values("def_power", ascending=False)[["club_name","power","def_power"]].head(5)

,club_name,power,def_power
84,Manchester City,85,85
109,Real Madrid,84,85
81,Liverpool,84,85
101,Paris Saint Germain,84,84
44,FC Bayern München,83,84


In [11]:
tfr_data = TFR(tpr_data, players_df, top_N_players=11)["data"]
tfr_data.sort_values("finishing", ascending=False)[["club_name","power","finishing"]].head(5)

,club_name,power,finishing
84,Manchester City,85,82
85,Manchester United,82,81
109,Real Madrid,84,81
70,Inter,83,80
143,Villarreal,80,80
